In [1]:
import pandas as pd
import sqlite3
import emoji
from collections import Counter

from matplotlib.pyplot import yscale, xscale, title, plot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
def read_sql():
    df = __read_sql()
    df = __select_taipei_area(df)
    df = __clean_raw_dataframe(df)
    return df

# read raw data from sql
def __read_sql():
    con = sqlite3.connect('../data/InstagramPost.sqlite')
    con2 = sqlite3.connect('../data/InstagramPost2.sqlite')
    con3 = sqlite3.connect('../data/InstagramPost3.sqlite')
    SQL = pd.read_sql_query('select * from InstagramPost', con)
    SQL2 = pd.read_sql_query('select * from InstagramPost_table2', con)
    SQL3 = pd.read_sql_query('select * from InstagramPost_table3', con)
    SQL4 = pd.read_sql_query('select * from InstagramPost_table4', con)
    SQL5 = pd.read_sql_query('select * from InstagramPost_table2', con2)
    SQL6 = pd.read_sql_query('select * from InstagramPost', con3)

    # concat SQL tables
    SQL = pd.concat([SQL, SQL2, SQL3, SQL4, SQL5, SQL6]).drop_duplicates()
    del SQL2, SQL3, SQL4, SQL5, SQL6
    SQL['datetime'] = pd.to_datetime(SQL.date, unit='s')
    instagram2016 = SQL[SQL.datetime.dt.year == 2016]
    del SQL
    return instagram2016

# select taipei area
def __select_taipei_area(df):
    # 信义松山
    lat_min, lat_max = 25.028791, 25.06338
    lng_min, lng_max = 121.54733, 121.59

    df['lat'] = df.lat.astype(float)
    df['lng'] = df.lng.astype(float)

    lat_bound = df.lat.between(lat_min, lat_max)
    lng_bound = df.lng.between(lng_min, lng_max)
    df = df[lat_bound & lng_bound]
    return df

# clean datafrome
def __clean_raw_dataframe(df):
    df = df[['location_id', 'location_name', 'lat', 'lng', 'caption', 'comments',
             'date', 'media_id', 'is_video', 'likes', 'owner_id', 'thumbnail_src',
             'display_src', 'datetime']].reset_index().drop(columns='index')
    return df

In [90]:
def emoji_tokenize(text):
    token = ''.join(c for c in text if c in emoji.UNICODE_EMOJI)
    return token

def create_emoji_column(df):
    df['emoji'] = df['caption'].apply(emoji_tokenize)
    return df

def count_emoji(text, freq):
    '''
    return the highest used emoji
    it only works for text including emoji only
    '''
    c = Counter(text)
    return c.most_common(freq)

def count_emoji_location(df, freq=1):
    '''
    group by location
    return df including the most used emoji
    '''
    df = create_emoji_column(df)
    gb = df.groupby('location_name')
    loc_df = pd.DataFrame(gb['emoji'].sum())
    loc_df['count'] = loc_df['emoji'].apply(lambda x: count_emoji(x, freq))
    return loc_df

# 1. Import Data

In [4]:
ig2016 = read_sql()

In [127]:
ig_emoji = count_emoji_location(ig2016, 10).reset_index()
ig_emoji.head()

,location_name,emoji,count
0,1/10 Cake,📍🏡💰🔆🔆💡😂🍵💗🥝🍊🍇🍈🌬☺🍰🍰🍰😘🎉💪😋📍🏡💰🔆🔆💡😂🍵💗👍💯🍋😋🎄😩😋🙈😋💦♥😳⭐🎉🎄...,"[(🍰, 23), (✨, 10), (😋, 9), (😍, 9), (😂, 8), (🍴,..."
1,10 Square Cafe,😘❤👼✨✨❤👼‼👍✨💛💚✨🎄✨✨💚❤💚🎉💚🎄🎁🎄💚👭😂😘💋❤🎄❤💚✨🎉🎉🎉💚🍾👍✨💚🎄💋🎄❤...,"[(❤, 72), (✨, 50), (😍, 24), (💕, 22), (🎄, 20), ..."
2,1001 Nights Taipei (一千零一夜水煙館),🤡🤡🤡🙄🍟👌😜😅😍😅🤔🤔🕴🍷💓🍵🎉❤🎄🎄🎄🚩🍷🍷🍷💕🚩🌬🌬🌬🙈🙈🙈🚩🍷🍸🍹⭐⭐⭐⭐⭐⭐✨✨👩...,"[(😂, 78), (❤, 71), (🍻, 46), (💕, 38), (🍸, 36), ..."
3,101 Night Club,🍻🍾❤,"[(🍻, 1), (🍾, 1), (❤, 1)]"
4,101♥ ♥ ♥,😏☕☕😭♥❤💖💖❓❓✨✨👭😘🌃🌠🌌🎆😜👭😂😂😂👉👈,"[(😂, 3), (☕, 2), (💖, 2), (❓, 2), (✨, 2), (👭, 2..."


# Count Emoji

In [128]:
def count_all_emoji(df):
    '''
    return the most common emoji in Taipei
    '''
    df = create_emoji_column(df)
    emoji_text = df.emoji.str.cat()
    counter = Counter(emoji_text)
    return counter

In [129]:
emoji_counter = count_all_emoji(ig2016)

In [131]:
emoji_counter.most_common(10)

[('❤', 62631),
 ('😂', 58929),
 ('😋', 38195),
 ('💕', 34469),
 ('😍', 33449),
 ('👍', 24454),
 ('😭', 19768),
 ('✨', 18925),
 ('😘', 16189),
 ('🎄', 15085)]

# Most used Emoji

In [132]:
# take out 10 most common emoji
# change rank into column
def change_rank_to_column(df, ktop):
    '''
    ktop -> how many column you need to build up
    '''
    ktop_counter = 0
    while ktop_counter < ktop:
        df['%s_emoji' % ktop_counter] = df['count'].apply(lambda x: x[ ktop_counter ][0] if ( len(x) > ktop_counter) else None)
        df['%s_value' % ktop_counter] = df['count'].apply(lambda x: x[ ktop_counter ][1] if ( len(x) > ktop_counter) else 0)
        ktop_counter += 1
    return df

In [133]:
ig_emoji = change_rank_to_column(ig_emoji, 5)

In [135]:
ig_emoji.head()

,location_name,emoji,count,0_emoji,0_value,1_emoji,1_value,2_emoji,2_value,3_emoji,3_value,4_emoji,4_value
0,1/10 Cake,📍🏡💰🔆🔆💡😂🍵💗🥝🍊🍇🍈🌬☺🍰🍰🍰😘🎉💪😋📍🏡💰🔆🔆💡😂🍵💗👍💯🍋😋🎄😩😋🙈😋💦♥😳⭐🎉🎄...,"[(🍰, 23), (✨, 10), (😋, 9), (😍, 9), (😂, 8), (🍴,...",🍰,23,✨,10,😋,9,😍,9,😂,8
1,10 Square Cafe,😘❤👼✨✨❤👼‼👍✨💛💚✨🎄✨✨💚❤💚🎉💚🎄🎁🎄💚👭😂😘💋❤🎄❤💚✨🎉🎉🎉💚🍾👍✨💚🎄💋🎄❤...,"[(❤, 72), (✨, 50), (😍, 24), (💕, 22), (🎄, 20), ...",❤,72,✨,50,😍,24,💕,22,🎄,20
2,1001 Nights Taipei (一千零一夜水煙館),🤡🤡🤡🙄🍟👌😜😅😍😅🤔🤔🕴🍷💓🍵🎉❤🎄🎄🎄🚩🍷🍷🍷💕🚩🌬🌬🌬🙈🙈🙈🚩🍷🍸🍹⭐⭐⭐⭐⭐⭐✨✨👩...,"[(😂, 78), (❤, 71), (🍻, 46), (💕, 38), (🍸, 36), ...",😂,78,❤,71,🍻,46,💕,38,🍸,36
3,101 Night Club,🍻🍾❤,"[(🍻, 1), (🍾, 1), (❤, 1)]",🍻,1,🍾,1,❤,1,None,0,None,0
4,101♥ ♥ ♥,😏☕☕😭♥❤💖💖❓❓✨✨👭😘🌃🌠🌌🎆😜👭😂😂😂👉👈,"[(😂, 3), (☕, 2), (💖, 2), (❓, 2), (✨, 2), (👭, 2...",😂,3,☕,2,💖,2,❓,2,✨,2


In [136]:
# drop value not include emoji
ig_emoji = ig_emoji.dropna(how='any').sort_values('1_value', ascending=False)

In [141]:
# drop value < 500
ig_emoji[ig_emoji['1_value'] > 300]

,location_name,emoji,count,0_emoji,0_value,1_emoji,1_value,2_emoji,2_value,3_emoji,3_value,4_emoji,4_value
94,BELLAVITA,⛲🎋‼🙏🎄🎄👊👊👊🗡😘❤👋🎆🌟😺❤❤😇😇😇😁🎆🎆🎆🌟🎆🎊😂😂💛🎈🎈🎈😌🎇🤣🎆🎉🎊🌟👩👩👧👏👏...,"[(🎄, 5219), (❄, 2933), (😂, 2268), (❤, 2094), (...",🎄,5219,❄,2933,😂,2268,❤,2094,⛄,1643
628,TAIPEI 101 MALL 台北 101 購物中心,🙏🙏🙏😜👭💕👀🙃🙃🎉🎉🎉👋🙋👭💕🎆✨✨✨😁😭😭🎉🎆😂😂👏👏👏👏👨👩👧👦❤👍🍻🎇💪😘🎇🎆🌠🎆🎆...,"[(🎄, 1804), (❤, 1721), (😂, 1513), (😍, 844), (🎉...",🎄,1804,❤,1721,😂,1513,😍,844,🎉,759
59,ATT 4 FUN,✨✨🎇😭👍👍👍💤🎆😪☺✨🥂🍾👋😂😆👋👐🎆🎉🎊🤘😪😭😭😭😭😭🎉🎉😍👍👍❤🎉🎆🐦🎄❤💛❤🎬🙈💕😈...,"[(😂, 1826), (❤, 1590), (💕, 852), (😭, 796), (😍,...",😂,1826,❤,1590,💕,852,😭,796,😍,789
996,信義區商圈,😵😴😪🤦♀🎉🎉🎉🎉❤💑😙🌹🎀🌺🎉🎆✌✌😀😆😆😆❤💪👭👭🎆🎊🎈🎉🍾🍾🙏😀😗🙌🙌👯👯👯😤😤😤👩👩...,"[(😂, 1384), (❤, 1321), (🎄, 950), (💕, 593), (😍,...",😂,1384,❤,1321,🎄,950,💕,593,😍,490
634,TICC 台北國際會議中心,❤🌈👍😂😂😂😂🎉😂💓❤🎤👻😭😭😭😭😭😭🎊🎈🎉😑😂👍👍👍🎉🎉🎉🎤👏👏👏🍻🍻🍻❤😍👪❤😜😍🎉🎉🎉...,"[(❤, 1458), (😂, 1223), (😭, 1125), (😍, 962), (💕...",❤,1458,😂,1223,😭,1125,😍,962,💕,848
2831,點點心台灣 Dimdimsum Taiwan,👉👈😍😋😋😋❤🍆🐷🐴🐵🐽🐷😋😋😋🐷😰🙈🐷😂😂😂😋😋😋😆🙁🐷🐽🐽🐽🐣🐣🐣🐷🐰✨✨✨😋😄🐷💛🐷💛...,"[(🐷, 3324), (😂, 1091), (😋, 1056), (🐽, 695), (❤...",🐷,3324,😂,1091,😋,1056,🐽,695,❤,683
1908,松山文創園區 SCCP Taipei,😴😴😴💕☀❤👍😍👋👋🅾💰💰💰🎆🎉❤❤❤❤❤❤❤🎆🌈🎉✨❤❤❤😌😂😂🙈🙈🔚💕💕💕💕💕💕💕💕💕💕...,"[(😂, 1194), (❤, 1026), (💕, 541), (😍, 530), (😭,...",😂,1194,❤,1026,💕,541,😍,530,😭,432
490,Okey Dokey,❤💛💚💙😝💕🎆🎊🍾🤤🤤☺☺👋👋😘😘😘❤🤤➡🎉🎉🎉☺😍💕🤗😂😂😂✌✌✌😗😗😗👍🍰🍕👍❤😂🚩🧀🍴...,"[(😋, 1038), (😂, 949), (❤, 774), (💕, 692), (🍕, ...",😋,1038,😂,949,❤,774,💕,692,🍕,673
785,YumYum Deli,😱👯😜😜😜😂😂😂➡😭🍻🍾🍹🌮😂👱♀👦👱♀😒😕😤😡🙄👦😘😂🤔❤❣✨🐼⛄🎄🎉💕🍹🐼📷🔥🔥👨👩👧👦...,"[(😂, 802), (❤, 755), (😋, 748), (💕, 680), (🐼, 6...",😂,802,❤,755,😋,748,💕,680,🐼,611
398,M-Palace 樂晴店,👉👉😜😜😜👉👉✌✌✌💓💓💓💓💓💛👇👇💕💕👇👇👇👇🙌🙌💲💲💰💓💓💓💓💓☎💑💑💑🌟🙇👉👉😜😜😜👉...,"[(💑, 1052), (💓, 712), (☎, 635), (👇, 602), (❤, ...",💑,1052,💓,712,☎,635,👇,602,❤,560


# Where is the "😍"

In [85]:
ig_emoji[ ig_emoji.most_emoji == '😍' ]

,location_name,emoji,count,most_emoji,most_value
273,HELLO KITTY Shabu-Shabu,😺😸😹😻😼😽🙀😋🙈😬😬❤❤❤😋😍😍▶▶💞💕👣😋😘🌸🌸🎀💞🎄👪📷🙄🙄😂😂👍👍👍⛽⛽⛽😍🎄💑😋😅...,"[(😍, 175)]",😍,175
638,TORUK - The First Flight,💸💳💶💷💨💰😂😂😂😘😎😇💊🎛🎧🎤💨💨💨💨💨💨💰💰💰🙏🙏🙏🙏🙏🙏🙏🙏🙏✌✌🐲🎍☄💫🎭🤹♂😁🐲🎍...,"[(😍, 128)]",😍,128
2311,繪本咖啡｜嬉々café,🎉🎉🎉😝🎆🎆🎆🍎🥕🥔🍳🍛💕🎂🎂😛😋😋😋🌟🌟🌟✌💕🎄📷😋😴🙈🙄😋💁💁💁💁💁💁😋😋😋😂☝💕💤💤💤...,"[(😍, 47)]",😍,47
2782,魚道生,😂😂😂😂😂🤓🍴🎄🍴🎄🎄👍🙈✌❤😭🍣🍲🍢🍣🍲🍢👏❤❤👏❤❤👏❤❤👏❤❤👏❤❤👏❤❤👏❤❤😭😭😭...,"[(😍, 40)]",😍,40
525,Peanuts專櫃-微風松高店,😎😆😏👍🐥☺☺☺🙋🚩😍🙃🤗🍦🍫✨✨✨💝🎄🎁💖😂🙂😂🎁😍😍😍😂😂😂↪↪👟😄😫🐶🐶🐶🐶🙁☔😎😆😆...,"[(😍, 27)]",😍,27
1805,旬採鮨処,🦀🐠🐟😍🐟👀😒💋😱🍣☺😝😝🍴🍣🍣😁🙌😋😋😁😁😋🐙🐟🦀🍤🍣🍙🍶🐣😋😝😝😝✌⭐⭐⭐⭐⭐👍😀😋😋😋...,"[(😍, 23)]",😍,23
2489,蓮香齋素食餐廳,🤗👯🍽😟😲😋🍴😍🎈🎈💖👍😚🤗🎉🎈💕💏💙🌹🍬🎈💕💖😍💖🌲🎄🎁🍬🎈💕😂🎄⛄💖🌳🌼🌸🎀🎁😘😍💍🎈💕...,"[(😍, 21)]",😍,21
1739,我樂制作所,👧😍😂😭😂😳😳🤔🍗🍖🍤🌯🌮😂😂🍽😆😆🍽🍽🍽🍗🎰🍝😁😁😁😋😋😋🌹👽🔻🕶👽🔻🈵🈵🉐🈚🈚🈚🎶👽🔻🍽...,"[(😍, 20)]",😍,20
465,N+n 喫早餐,🍴☕🍳😂🤗😋➕☕💼☀😆🤗🐣🐣😭👉😍🤗💩😂👍👍🤘🍔🍟🍊🍐🌧😂🍴🍔🌯🍕🌮🙋💕💞💞🍽☀🙋♂☑💕💕😁...,"[(😍, 19)]",😍,19
1607,小間咖啡酒館 Le Petit Café,😄😄😄😊😊😊🍷🍸🍾😊😊😊📷✨🌙🍴😘😘😘👯🍷🍾💕😍😍😍😍🍾🍾🍾🍾🍾🍾🌧🌧🌧🌧🌧🌧🌧🌧🤗🤗🤗🤗🤗...,"[(😍, 17)]",😍,17


# Where is the "😭"

In [86]:
ig_emoji[ ig_emoji.most_emoji == '😭' ]

,location_name,emoji,count,most_emoji,most_value
2536,誠品電影院 eslite art house,👍😄😄🎬😆😱👍👍👍👍👍👍👍👍👍👍☺🎶🎬🤓☺☝😂😍🎦😎👍👍👍💪🎄😭😭👀😳😳🐱🐈🔥🌚🎥🔨💣🔪💔❤...,"[(😭, 66)]",😭,66
1376,國泰綜合醫院,👼😂🏥😑😞🎅🎁🎄🤣🤤😏👀😨👀❤🎁🎁😂😆😆💕😭😭😊✔😏🎄😩😆✌✍💉😏😣😷😣😑💩😙😂😂😂😂😂😏💤...,"[(😭, 61)]",😭,61
2654,長庚紀念醫院台北分院,💃😭❤🙈💉💉💉🎵😳😘❤😷😷😷🤒🤒🤒🤕🤕🤕😢😢😢😭😭😭😭😭😭❤😭☁😷😥😥😥😭😭😭💓💓😂😂👨👶👩...,"[(😭, 58)]",😭,58
2038,永春市場,😍😆☃😋😋😋💕👍😳😭😭😭🐣😎❤🤔🤔💜💛❤💙💚💗👊👊👊👊😭😭😭😭🗻🗻🗻🗻🗻💕❤🐟😄😄😍😍😑😑😑...,"[(😭, 36)]",😭,36
1673,後台很硬,🎉🌹✨😭😭💓😘🙏🙏🙏😂😂😂😅😅😭😭😭😭😭😭👉😑🎃🎃🎃😆😆😆😳😳😳😳🤘😆😆😆😆😆😆😆😆😆😆😆😆...,"[(😭, 25)]",😭,25
2636,銅錵鍋日式涮涮鍋,❤🍴🎄😋✌🐽😙🍴🤓🤓🤓🤓😂🎉🎄👍✨😍❤😁😋😋😈👍🙌❤🐷🍲🍖🌽🏡💕😂😂😂😂😍☺💕🍴😭😭😭😂😂😂...,"[(😭, 21)]",😭,21
2043,永春高中英華樓,💕🌲🌲👉✨🎉🎉🎉💛😄🎬😢😩😭☁😂😂💕👦💛👧😍😍😈💋💋💋👬🎉👍👍😂😂😂😎😭😂🔥😘😁😂😂😍😍💕✨...,"[(😭, 20)]",😭,20
1144,博仁醫院,💗❤😽😽😞😭😭😭😭😭😭😭😭🙄💕💕💕😓😓😓👃😭😭😭😩😩😩😂😱😱🤓😂😂💪😂😂😂💩😱😙😙😙🙏😩💉💉...,"[(😭, 17)]",😭,17
894,中坡耳鼻喉科診所,😭😭😭😭😭😭😭💔😭😭😭😭😭😭😭😭😂💔💔💔🙄🙄🙄😷😓🤕🤒😴😷💀😥,"[(😭, 15)]",😭,15
724,VOEZ Cafe,😂❤💕💕💕🎠☕😁😁😁😁😋😋😋✔💔👍✨📚🎹😱⭐♥♥♥😂🌞😁💪💪💪✨🍵🙌🙌🙌🙌😭😂❤❤❤❤❤😭😭...,"[(😭, 15)]",😭,15
